In [124]:
from pywr.core import Model
from pywr.notebook import draw_graph
import matplotlib.pyplot as plt
import os
from pywr.domains.groundwater import KeatingAquifer
from pywr.parameters.groundwater import KeatingStreamFlowParameter
from pywr.core import Model, Input, Output, Link, Storage, Node, Catchment
from pywr.recorders import (
    NumpyArrayNodeRecorder,
    NumpyArrayLevelRecorder,
    NumpyArrayStorageRecorder,
)
import pandas
import numpy as np
import pytest

def createAquifer(model,name):
    num_streams = 1
    num_additional_inputs = 0
    stream_flow_levels = [[1.0, 10.0]]  # m
    transmissivity = [100, 2000]  # m2/d
    transmissivity = [t * 0.0001 for t in transmissivity]  # m3 to Ml
    coefficient = 1  # no units
    storativity = [0.000005]  # % este parámetro es clave
    levels = [1.0, 10.0]  # m
    area = 5000 * 5000  # m2
    
    aqfer=KeatingAquifer(
        model,
        name,
        num_streams,
        num_additional_inputs,
        stream_flow_levels,
        transmissivity,
        coefficient,
        levels,
        area=area,
        storativity=storativity)

    aqfer.initial_level=1
    
    return aqfer

def connectAquifer(modelo,acuifero,nodosRecarga,nodosDescarga):
    for nodoR in nodosRecarga:
        model.nodes[nodoR].connect(acuifero)
    
    for nodoD in nodosDescarga:
        acuifero.connect(modelo.nodes[nodoD],from_slot=0)
    
    return modelo,acuifero
    

if __name__ == "__main__":
    
    path="nuble.json"
    model = Model.load(path)
    
#     crear objetos acuíferos
    aqferNiquen=createAquifer(model,'AC01_NIQ')
    aqferChan=createAquifer(model,'AC02_CHA')
    aqferNuble=createAquifer(model,'AC03_NUB')
    
#     conectar nodos existentes al acuifero Ñiquen
    model,aqferNiquen=connectAquifer(model,aqferNiquen,
['REMZemitaSub','REMVirguinJFRSub','REMSanPedroSub'],
['abstraccionAqNiq','ZemitaMix','VirguinJFRMix','deficitSanPedro'])

#     conectar nodos existentes al acuifero Changaral
    model.nodes['REMGYMSub'].connect(aqferChan)
    aqferChan.connect(model.nodes['abstraccionAqCha'],
from_slot=0)
    aqferChan.connect(model.nodes['GreeneYMairaMix'],
                            from_slot=0)
    model.nodes['REMMunicipalSub'].connect(aqferChan)
    aqferChan.connect(model.nodes['deficitMunicipal'],
                            from_slot=0)
    model.nodes['REMSACSub'].connect(aqferChan)
    model.nodes['REMCSSub'].connect(aqferChan)
    aqferChan.connect(model.nodes['deficitCS'],
                            from_slot=0)
    model.nodes['REMRanchilloSub'].connect(aqferChan)
    aqferChan.connect(model.nodes['deficitRanchillo'],
                            from_slot=0)
    model.nodes['REMGaonaPeralesSub'].connect(aqferChan)
    aqferChan.connect(model.nodes['deficitGaonaPerales'],
                            from_slot=0)
    model.nodes['REMArrauNiquenSub'].connect(aqferChan)
    aqferChan.connect(model.nodes['deficitArrauNiquen'],
                            from_slot=0)
    model.nodes['REMLurinSilvaSub'].connect(aqferChan)
    aqferChan.connect(model.nodes['deficitLurinSilva'],
                            from_slot=0)
    model.nodes['REMSantaSaraSub'].connect(aqferChan)
    aqferChan.connect(model.nodes['deficitSantaSara'],
                            from_slot=0)
    model.nodes['REMPomuyetoAltoSub'].connect(aqferChan)
    aqferChan.connect(model.nodes['deficitPomuyetoAlto'],
                            from_slot=0)
    model.nodes['REMSantaMartaSub'].connect(aqferChan)
    aqferChan.connect(model.nodes['deficitSantaMarta'],
                            from_slot=0)
    
#     conectar nodos existentes al acuifero Ñuble
    model.nodes['REMCollicoSub'].connect(aqferNuble)
    model.nodes['REMChacayalOSub'].connect(aqferNuble)
    model.nodes['REMChacayalPSub'].connect(aqferNuble)
    aqferNuble.connect(model.nodes['abstraccionAqNub'],
from_slot=0)
    aqferNuble.connect(model.nodes['deficitCollico'],
                            from_slot=0)
    aqferNuble.connect(model.nodes['deficitChacayalO'],
                            from_slot=0)
    aqferNuble.connect(model.nodes['deficitChacayalP'],
                            from_slot=0)
    model.nodes['REMMuticuraSub'].connect(aqferNuble)
    aqferNuble.connect(model.nodes['deficitMuticura'],
                            from_slot=0)
        
        
#     conectar los nodos de déficit que faltan
    model.nodes['deficitCollico'].connect(model.nodes['CollicoMix'])
    model.nodes['deficitChacayalO'].connect(model.nodes['ChacayalOMix'])    
    model.nodes['deficitChacayalP'].connect(model.nodes['ChacayalPMix'])
    model.nodes['deficitMunicipal'].connect(model.nodes['MunicipalMix'])
    model.nodes['deficitCS'].connect(model.nodes['CaroSaldanaMix'])
    model.nodes['deficitRanchillo'].connect(model.nodes['RanchilloMix'])    
    model.nodes['deficitGaonaPerales'].connect(model.nodes['GaonaPeralesMix'])    
    model.nodes['deficitSanPedro'].connect(model.nodes['SanPedroMix'])    
    model.nodes['deficitArrauNiquen'].connect(model.nodes['ArrauNiquenMix'])    
    model.nodes['deficitLurinSilva'].connect(model.nodes['LurinSilvaMix'])    
    model.nodes['deficitMuticura'].connect(model.nodes['MuticuraMix'])    
    model.nodes['deficitSantaSara'].connect(model.nodes['SantaSaraMix'])   
    model.nodes['deficitPomuyetoAlto'].connect(model.nodes['PomuyetoAltoMix'])
    model.nodes['deficitSantaMarta'].connect(model.nodes['SantaMartaMix'])

#     correr modelo
    model.solver.name="glpk-edge"
    stats = model.run()
    print(stats)
    
    plt.figure()
    draw_graph(model,labels=True,attributes=True,
               width=950,height=950)
    plt.show()


Model executed 1 scenarios in 2.7 seconds, running at 598.2 timesteps per second.


<IPython.core.display.Javascript object>

<Figure size 640x480 with 0 Axes>

In [98]:
# bocatoma 3
print((1000-model.nodes["bocatoma3"].flow)/2)
#validar el balance de masa en canal Virguin
print(model.nodes["FRVirguin"].flow)
print(model.nodes["REMVirguinJFR"].flow)
#validar balance de masa en canal Collico
print(model.nodes["CollicoSup"].flow)
print(model.nodes["CollicoMix"].flow)
print(model.nodes["REMCollico"].flow)
print(model.nodes["deficitCollico"].flow)
print(model.nodes["FRCollico"].flow)
print(model.nodes["bocatoma3"].flow)
print(model.nodes["bocatoma4"].flow)


[14.37571429]
[28.57142857]
[0.]
[14.28571429]
[14.28571429]
[0.]
[0.]
[28.75142857]
[971.24857143]
[942.49714286]


In [2]:
# from pywr.notebook import to_html
# to_html(m,height=800,width=800,labels=True,attributes=True,title='Modelo Cuenca del Río Huasco')

In [10]:
from pywr.notebook import pywr_model_to_d3_json
d3json=pywr_model_to_d3_json(model)
import json
with open("./d3.json", "w") as fp:
    json.dump(d3json,fp) 